K Nearest Neighbors is an algorithm which can be used for regression or classification. We need numerical data. We think of the observations (rows) as points in $n$-dimensional space. Given a training set and a point $X$ for prediction, we calculate the $k$ nearest points from the training set to $X$ (hence the name of the algorithm). For regression, we guess the average of the target values for the $k$ nearest neighbors. For classification, we hold a majority vote.

There are tie conditions to consider: what happens (in regression or classification) when we have a tie for the $k^\text{th}$ closest point, and what happens in classification when we have a tied vote?

In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from itertools import product
from multiprocessing import Pool
from os import cpu_count
cores = cpu_count() # get the number of CPUs

df = pd.read_csv('wine.csv')

features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'color']
target = ['quality']

x_tr,x_te,y_tr,y_te = train_test_split(df[features],df[target],
                                       test_size = 0.2)
x_tr,x_va,y_tr,y_va = train_test_split(x_tr,y_tr,
                                       test_size = 0.5)

We can get the (euclidean) distance between two rows by using linalg.norm.

In [4]:
def dist(row1,row2):
    return(np.linalg.norm(row1-row2))

dist(x_tr.iloc[0],x_tr.iloc[1]) # distance between row1 and row2 of x_tr

46.93172771495207

To make a prediction using KNN, there's no pre-training necessary (although we'll see how some organization of the data is crucial for reasonable performance). We do need to get the $k$ nearest neighbors to a given point. One way to do that is to compute the distance to all the training points, then sort.

In [6]:
def get_distances(inpt):
    i,x_tr,y_tr,x_pred,k = inpt
    # I packaged the inputs for multiprocessing.
    # accepts: i: index (iloc) of a row in the prediction inputs x_pred
    #          x_tr: the training features
    #          y_tr: the training target values
    #          x_pred: the inputs needing predictions
    #          k: a positive integer
    # returns: the average of the k training target values of rows closest to the given row
    res = []
    for j,row in x_tr.iterrows():
        res.append((j,dist(row,x_pred.iloc[i])))
    res.sort(key = lambda x:x[1])
    res = res[:k]
    return((i,np.mean([y_tr.loc[x[0]] for x in res])))

In [10]:
# example usage
get_distances([2,x_tr,y_tr,x_va,5])

(2, 6.2)

In [12]:
# get predictions for the first N points in the validation set
N = 100
inpts = product(range(N),[x_tr],[y_tr],[x_va],[5])
# inpts = product(range(len(x_va)),[x_tr],[y_tr],[x_va],[5])
with Pool(processes = cores-1) as p:
    res = p.map(get_distances,inpts)
# just to be sure that map returns things in order, I included indices for sorting
preds = [_[1] for _ in res]

In [15]:
# compute the R^2 for our KNN model
1 - np.sum((y_va[:N]-np.array(preds).reshape(-1,1))**2)/\
np.sum((y_va[:N]-np.mean(y_va[:N]))**2)

/home/briggsc1-erau.edu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


quality   -0.054966
dtype: float64

This isn't great. Can you think of any reasons why this might not be great? If you guessed "the data should be normalized," please give yourself a gold star.

In [18]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()
scaler.fit(x_tr)

# get predictions for the first 100 points in the validation set
x_tr_n = pd.DataFrame(scaler.transform(x_tr))
x_tr_n.columns = features
x_tr_n.index = x_tr.index
x_va_n = pd.DataFrame(scaler.transform(x_va))
x_va_n.columns = features
x_va_n.index = x_va.index
inpts = product(range(N),[x_tr_n],[y_tr],[x_va_n],[5])
# inpts = product(range(len(x_va)),[x_tr],[y_tr],[x_va],[5])
with Pool(processes = cores-1) as p:
    res = p.map(get_distances,inpts)

preds = np.array([_[1] for _ in res]).reshape(-1,1)
1 - np.sum((y_va[:N]-preds)**2)/\
    np.sum((y_va[:N]-np.mean(y_va[:N]))**2)

/home/briggsc1-erau.edu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


quality    0.090568
dtype: float64

Something we might try to save time is to, rather than compute euclidean distances, compute their squares (that is, skip taking square roots). However, np.linalg.norm is well optimized, so this offers us no time savings:

In [26]:
%%timeit
np.linalg.norm(x_tr.iloc[0,]-x_tr.iloc[1,])

203 µs ± 762 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
%%timeit
np.mean(np.square(x_tr.iloc[0,]-x_tr.iloc[1,]))

316 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Something we might try to improve accuracy: check the correlation of each feature with the target and weight distances by some function of the correlation. Rather than defining our own distance function, which would slow things down vs using the optimized linalg distance, we can just scale the data.

In [31]:
# correlations with quality
df.corr()['quality']

fixed acidity          -0.076743
volatile acidity       -0.265699
citric acid             0.085532
residual sugar         -0.036980
chlorides              -0.200666
free sulfur dioxide     0.055463
total sulfur dioxide   -0.041385
density                -0.305858
pH                      0.019506
sulphates               0.038485
alcohol                 0.444319
quality                 1.000000
color                  -0.119323
Name: quality, dtype: float64

In [32]:
# get the absolute value of the correlations with quality
abs_cor = abs(df.corr()['quality'].loc[features,])

In [34]:
x_tr_w = x_tr_n.apply(lambda row:row*abs_cor,axis = 1)
x_va_w = x_va_n.apply(lambda row:row*abs_cor,axis = 1)

In [35]:
inpts = product(range(N),[x_tr_w],[y_tr],[x_va_w],[5])
with Pool(processes = cores-1) as p:
    res = p.map(get_distances,inpts)

preds = np.array([_[1] for _ in res]).reshape(-1,1)
1 - np.sum((y_va[:N]-preds)**2)/\
    np.sum((y_va[:N]-np.mean(y_va[:N]))**2)

/home/briggsc1-erau.edu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


quality    0.166771
dtype: float64

Of course sklearn has KNN.

In [38]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()

In [42]:
%%timeit
knn.fit(scaler.transform(x_tr),y_tr)
knn.score(scaler.transform(x_va),y_va)

61.3 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Notes on this algorithm: the default number of neighbors is 5 (and it can be changed with parameter n_neighbors). Their contributions are by default weighted equally, but this can be altered to account for distance with parameter weights = 'distance'.

How does it run so quickly? It uses $k$-D trees. Time out to discuss $k$-D trees on the whiteboard, including the nearest neighbor search algorithm.